In [18]:
# this worksheet is meant to accompany the paper:
# "Uniquely optimal codes of low complexity are symmetric"
# by Christopher Cox, Emily J. King, Dustin G. Mixon, and Hans Parshall
#
# inquiries may be sent to Hans at hans.parshall@wwu.edu

In [10]:
# imports
import numpy as np
import itertools as it

# common functions

# remove duplicates from a sequence of vectors
def sequence_to_set(seq):
    output = []
    for v in seq:
        if any(np.array_equal(v, w) for w in output):
            continue
        else:
            output.append(v)
    return output

In [11]:
# build D4 objects

# basis for D4
D4_B = np.array([(1,1,0,0),(1,-1,0,0),(0,1,-1,0),(0,0,1,-1)])

# return fundamental domain representative
def D4_canonicalize(v):
    Binv = np.linalg.inv(D4_B.T)
    w = np.array(np.divmod(np.dot(Binv,v.T),1)[1])
    return np.dot(D4_B.T,w)

D4_zero_holes = [(1,0,0,0),(-1,0,0,0),(0,1,0,0),(0,-1,0,0),(0,0,1,0),(0,0,-1,0),(0,0,0,1),(0,0,0,-1),(1/2,1/2,1/2,1/2),(-1/2,1/2,1/2,1/2),(1/2,-1/2,1/2,1/2),(1/2,1/2,-1/2,1/2),
          (1/2,1/2,1/2,-1/2),(-1/2,-1/2,1/2,1/2),(-1/2,1/2,-1/2,1/2),(-1/2,1/2,1/2,-1/2),
          (1/2,-1/2,-1/2,1/2),(1/2,-1/2,1/2,-1/2),(1/2,1/2,-1/2,-1/2),(-1/2,-1/2,-1/2,1/2),
          (-1/2,-1/2,1/2,-1/2),(-1/2,1/2,-1/2,-1/2),(1/2,-1/2,-1/2,-1/2),(-1/2,-1/2,-1/2,-1/2)]

D4_zero_holes = [np.array(v) for v in D4_zero_holes]

# deep holes in the fundamental domain
D4_fund_holes = sequence_to_set([D4_canonicalize(v) for v in D4_zero_holes])

# detect if a vector is a deep hole
def is_D4_hole(v):
    w = D4_canonicalize(v)
    if any(np.array_equal(w,x) for x in D4_fund_holes):
        return True
    else:
        return False

H = D4_fund_holes
n = len(H)
D4_hole_graph = Graph(n)
for i in range(n):
    for j in range(i+1,n):
        if is_D4_hole(H[i] - H[j]):
            D4_hole_graph.add_edge(i,j)

D4_min_vecs = []
# (1,1,0,0), binomial(4,2)*2^2 signed permutations
for p in it.product([-1,1,0],repeat=4):
    if sum([abs(p[i]) for i in range(4)]) == 2:
        D4_min_vecs.append(np.array(p))

In [12]:
len(D4_fund_holes)

3

In [13]:
# build E8 objects

# basis for E8
E8_B = np.array([(2,0,0,0,0,0,0,0),
    (-1,1,0,0,0,0,0,0),
    (0,-1,1,0,0,0,0,0),
    (0,0,-1,1,0,0,0,0),
    (0,0,0,-1,1,0,0,0),
    (0,0,0,0,-1,1,0,0),
    (0,0,0,0,0,-1,1,0),
    (1/2,1/2,1/2,1/2,1/2,1/2,1/2,1/2)])

# return fundamental domain representative
def E8_canonicalize(v):
    Binv = np.linalg.inv(E8_B.T)
    w = np.array(np.divmod(np.dot(Binv,v.T),1)[1])
    return np.dot(E8_B.T,w)

E8_zero_holes = []

# (2,0,0,0,0,0,0,0), 8*2 signed permutations
for i in range(8):
    for s in [-1,1]:
        add_this = np.array([0]*8)
        add_this[i] = s*2
        E8_zero_holes.append(add_this)

# (1,1,1,1,0,0,0,0), binomial(8,4)*2^4 signed permutations
for p in it.product([-1,1,0],repeat=8):
    if sum([abs(p[i]) for i in range(8)]) == 4:
        E8_zero_holes.append(np.array(p))

# (3/2,1/2,1/2,1/2,1/2,1/2,1/2,1/2), 8*2^7 signed permutations with an "odd sign"
for i in range(8):
    for s in it.product([-1,1],repeat=8):
        add_this = ([s[j] for j in range(8)])
        if product(add_this) < 0:
            for j in range(8):
                if i == j:
                    add_this[j] = s[j]*(3/2)
                else:
                    add_this[j] = s[j]*(1/2)
            E8_zero_holes.append(np.array(add_this))

E8_zero_holes = [(1/2)*v for v in E8_zero_holes]

# deep holes in the fundamental domain
E8_fund_holes = sequence_to_set([E8_canonicalize(v) for v in E8_zero_holes])

# detect if a vector is a deep hole
def is_E8_hole(v):
    w = E8_canonicalize(v)
    if any(np.array_equal(w,x) for x in E8_fund_holes):
        return True
    else:
        return False

H = E8_fund_holes
n = len(H)
E8_hole_graph = Graph(n)
for i in range(n):
    for j in range(i+1,n):
        if is_E8_hole(H[i] - H[j]):
            E8_hole_graph.add_edge(i,j)

E8_min_vecs = []

# (1,1,0,0,0,0,0,0), binomial(8,2)*2^2 signed permutations
for p in it.product([-1,1,0],repeat=8):
    if sum([abs(p[i]) for i in range(8)]) == 2:
        E8_min_vecs.append(np.array(p))

# (1/2,1/2,1/2,1/2,1/2,1/2,1/2,1/2), 2^7 signed permutations with an "even sign"
for s in it.product([-1,1],repeat=8):
    add_this = ([s[j] for j in range(8)])
    if product(add_this) > 0:
        for j in range(8):
            add_this[j] = s[j]*(1/2)
        E8_min_vecs.append(np.array(add_this))

In [14]:
# test with E8
print(E8_hole_graph.order())
print(E8_hole_graph.size())
M = E8_hole_graph.cliques_maximal()
print(len(M[0])) # clique is of size 15, including the origin yields 16
print(len(M))

135
4725
15
270


In [15]:
M2 = E8_hole_graph.cliques_maximum()
print(len(M2))

270


In [16]:
def verts_to_points(V):
    return [np.array([0]*8)] + [E8_fund_holes[j] for j in V]

def points_to_verts(H):
    V = []
    for h in H:
        if np.array_equal(h, np.array([0]*8)) == False:
            V += [[list(a) for a in E8_fund_holes].index(list(h))]
    V.sort()
    return V

def reflect_x(x,root):
    Px = np.dot(root,x)*root / np.dot(root,root)
    Rx = x - 2*Px
    return Rx

def reflect_C(clique,root):
    H = E8_fund_holes
    return [reflect_x(H[j],root) for j in clique]

In [17]:
trials = 10000
seen = [0]*270
C = M[0]
seen[0] = 1
for trial in range(trials):
    P = verts_to_points(C)
    rand = randint(0,len(E8_min_vecs) - 1)
    r = E8_min_vecs[rand]
    R = [reflect_x(p,r) for p in P]
    FR = [E8_canonicalize(dude) for dude in R]
    new_C = points_to_verts(FR)
    seen[M.index(new_C)] = 1
    C = new_C
    total = sum(seen)
    if total == len(M):
        print(trial,sum(seen))
        print("we did it!")
        break
    elif trial % 100 == 0:
        print(trial,sum(seen))

0 2


100 77


200 128


300 171


400 200


...

1800 269


1900 269


2000 269


2100 269


2159 270
we did it!
